# **DataWarehouse Data Transform**
1. 동일한 컬럼 가지고 있는 데이터끼리 분류하여 '관광지 분류' 컬럼으로 생성  
2. 축제행사, 공연 : 날짜 지난 관광지 제거
3. 필터링 기능에 포함시킬 컬럼 전처리
   - 요금정보 ['이용요금', '입장료', '상세정보'] : 유료/무료 형태로
   - 유모차 대여 여부 ['유모차 대여 여부'] : 없음/불가/가능
   - 애완동물 동반 가능 여부 ['애완동물 동반 가능 여부'] : 없음/불가/가능
4. 관람/체험 연령 관련 컬럼 전처리
5. 명칭 컬럼 Null 데이터 제거
6. 불필요한 데이터 제거
7. 인덱스 생성

## **Packages**

In [1]:
# packages
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from hdfs import InsecureClient
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 

In [2]:
# matplotlib 한글폰트 
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='D2Coding')
       
# rc('font', family='NanumGothic') # 나눔폰트 사용시

## **DW에 있는 Data 사용하기 위한 Hdfs 클라이언트 객체 생성**

In [3]:
client = InsecureClient('http://localhost:9870', user='lab09')

## **Data 불러오기**
- travel_data_dropped.json : 불필요한 컬럼을 제거한 raw data 

In [4]:
df_tmp = spark.read.json('/dw_data/travel_data_dropped.json', encoding='utf-8')

In [5]:
df_tmp.printSchema()

root
 |-- 개요: string (nullable = true)
 |-- 개장기간: string (nullable = true)
 |-- 공연시간: string (nullable = true)
 |-- 관람가능연령: string (nullable = true)
 |-- 관람소요시간: string (nullable = true)
 |-- 명칭: string (nullable = true)
 |-- 분류: string (nullable = true)
 |-- 상세정보: string (nullable = true)
 |-- 쉬는날: string (nullable = true)
 |-- 애완동물 동반 가능 여부: string (nullable = true)
 |-- 우편번호: string (nullable = true)
 |-- 유모차 대여 여부: string (nullable = true)
 |-- 이용시간: string (nullable = true)
 |-- 이용시기: string (nullable = true)
 |-- 이용요금: string (nullable = true)
 |-- 입장료: string (nullable = true)
 |-- 전화번호: string (nullable = true)
 |-- 주소: string (nullable = true)
 |-- 주차시설: string (nullable = true)
 |-- 주차요금: string (nullable = true)
 |-- 지역: string (nullable = true)
 |-- 체험가능연령: string (nullable = true)
 |-- 체험안내: string (nullable = true)
 |-- 행사시작일: long (nullable = true)
 |-- 행사종료일: long (nullable = true)



## **1️⃣ 동일한 컬럼 가지고 있는 데이터끼리 분류하여 '관광지 분류' 컬럼으로 생성**

[관광지 분류]
- 축제공연행사
    - 축제
    - 공연행사 
- 문화시설
    - 문화시설 
- 레포츠
    - 레포츠 
- 인문관광지
    - 휴양관광지
    - 역사관광지
    - 자연관광지
    - 관광자원 

In [6]:
df_tmp.select("분류").distinct().show()

+----------+
|      분류|
+----------+
|  문화시설|
|자연관광지|
|역사관광지|
|      축제|
|  공연행사|
|  관광자원|
|휴양관광지|
|    레포츠|
+----------+



In [7]:
# package
from pyspark.sql import functions as F


# '분류' values <-> '관광지 분류' values 매핑시키기
def map_classification(category):
    if category in ['축제', '공연행사']:
        return '축제공연행사'
    elif category == '문화시설':
        return '문화시설'
    elif category == '레포츠':
        return '레포츠'
    elif category in ['휴양관광지', '역사관광지', '자연관광지', '관광자원']:
        return '인문관광지'
    else:
        return 'Unknown'

mapping_udf = F.udf(map_classification)


# 매핑된 결과 '관광지 분류'로 생성
df_result = df_tmp.withColumn('관광지 분류', mapping_udf(df_tmp['분류']))

In [8]:
# 확인
df_result.filter(df_result['관광지 분류'] == '인문관광지').select('관광지 분류', '분류').distinct().show()
df_result.filter(df_result['관광지 분류'] == '레포츠').select('관광지 분류', '분류').distinct().show()
df_result.filter(df_result['관광지 분류'] == '축제공연행사').select('관광지 분류', '분류').distinct().show()
df_result.filter(df_result['관광지 분류'] == '문화시설').select('관광지 분류', '분류').distinct().show()

+-----------+----------+
|관광지 분류|      분류|
+-----------+----------+
| 인문관광지|자연관광지|
| 인문관광지|역사관광지|
| 인문관광지|  관광자원|
| 인문관광지|휴양관광지|
+-----------+----------+

+-----------+------+
|관광지 분류|  분류|
+-----------+------+
|     레포츠|레포츠|
+-----------+------+

+------------+--------+
| 관광지 분류|    분류|
+------------+--------+
|축제공연행사|공연행사|
|축제공연행사|    축제|
+------------+--------+

+-----------+--------+
|관광지 분류|    분류|
+-----------+--------+
|   문화시설|문화시설|
+-----------+--------+



## **2️⃣ '축제공연행사' : 날짜 지난 관광지 제거**
- '행사종료일' 기준으로 서비스 출시일 2월8일 이전에 종료되는 데이터 제거
- '행사종료일' = LongType (=Represents 8-byte signed integer numbers)

In [9]:
# 20240208 이후 데이터만 저장 
df_filtered = df_result.filter(df_result.행사종료일 >= 20240208)

In [10]:
# 확인

## 제거하고 남은 데이터 개수 확인 
print('제거하고 남은 데이터 개수: ', df_filtered.count())

## 행사종료일이 '2023'으로 시작하는 데이터 (null 나와야 함)
df_filtered.select('행사종료일').filter(df_filtered.행사종료일.like('2023%')).count()

## 행사종료일이 '2024'으로 시작하는 데이터 (0208 이후 데이터만 포함되어야 함)
df_filtered.select('행사종료일').filter(df_filtered.행사종료일.like('2024%')).distinct().show()

제거하고 남은 데이터 개수:  36


0

+----------+
|행사종료일|
+----------+
|  20241222|
|  20240331|
|  20240229|
|  20240324|
|  20240228|
|  20241205|
|  20240721|
|  20241231|
|  20240225|
|  20240505|
|  20241207|
|  20240317|
|  20240303|
|  20240720|
|  20240226|
|  20240218|
|  20240901|
|  20240509|
|  20240212|
|  20240413|
+----------+
only showing top 20 rows



In [11]:
# 원본데이터의 '축제공연행사' 데이터 중 위 df_filtered(0208 이후 데이터) 만 남기기

## merge 할 때 구분하기 위해 새로운 컬럼 생성 
df_fest = df_filtered.withColumn("구분", lit("yes"))
df_result_new = df_result.withColumn("구분", lit(None))

## 두 데이터 합치기
merged_df = df_result_new.union(df_fest)

## 합친 데이터에서 "구분" 컬럼 None 행 제거
filtered_df = merged_df.filter(
    (merged_df['관광지 분류'] != "축제공연행사") | (merged_df['구분'].isNotNull()))

## "구분" 컬럼 drop
df_fin = filtered_df.drop("구분")

In [12]:
# 확인

## 데이터 개수 확인 
print('데이터 개수: ', df_fin.count())

## 행사종료일이 '2024'으로 시작하는 데이터 (0208 이후 데이터만 포함되어야 함)
df_fin.select('행사종료일').filter(df_fin.행사종료일.like('2024%')).distinct().show()

데이터 개수:  3646
+----------+
|행사종료일|
+----------+
|  20241222|
|  20240331|
|  20240229|
|  20240324|
|  20240228|
|  20241205|
|  20240721|
|  20241231|
|  20240225|
|  20240505|
|  20241207|
|  20240317|
|  20240303|
|  20240720|
|  20240226|
|  20240218|
|  20240901|
|  20240509|
|  20240212|
|  20240413|
+----------+
only showing top 20 rows



## **3️⃣ 필터링 기능에 포함시킬 컬럼 전처리**
- 요금정보 ['이용요금', '입장료', '상세정보'] : 유료/무료/알수없음
- 유모차 대여 여부 ['유모차 대여 여부'] : 없음/불가/가능
- 애완동물 동반 가능 여부 ['애완동물 동반 가능 여부'] : 없음/불가/가능

### 📍 요금정보 ['이용요금', '입장료', '상세정보'] : 유료/무료 형태로
- '유/무료 여부' 컬럼 새로 생성

In [13]:
# # '유/무료 여부' 컬럼을 생성하고, 모든 행을 'X'로 초기화
# df_fin = df_fin.withColumn('유/무료 여부', lit('X'))

In [14]:
# ### 1. 관광지 분류 == '레포츠'
# from pyspark.sql.functions import coalesce

# # '결제정보' 컬럼 생성
# df_fin = df_fin.withColumn('결제정보', when(col('관광지 분류') == '레포츠', 
#                                         concat_ws(' ', coalesce(col('이용요금').cast('string'), ''), 
#                                                   coalesce(col('상세정보'), ''))).otherwise(col('결제정보')))

# # '무료' 및 '유료' 여부 확인
# keywords = ['무료', r'\d+천원', r'\d+원']
# filtered_rows = expr("rlike(결제정보, '{}') AND NOT (rlike(결제정보, '{}') OR rlike(결제정보, '{}'))".format('|'.join(keywords), r'\d+천원', r'\d+원'))

# paid_kw = [r'\d+천원', r'\d+원']
# paid_rows = expr("rlike(결제정보, '{}') OR rlike(결제정보, '{}')".format(*paid_kw))



# # '유/무료 여부' 컬럼 생성
# df_fin = df_fin.withColumn('유/무료 여부', when((col('관광지 분류') == '레포츠') & filtered_rows, '무료')
#                                 .when((col('관광지 분류') == '레포츠') & paid_rows, '유료')
#                                 .otherwise('알 수 없음'))
# # '결제정보' 컬럼 제거
# df_fin = df_fin.drop('결제정보')

# # 결과 표시
# df_fin.select('유/무료 여부').show()

In [15]:
# ### 2. 관광지분류 == '인문관광지'

# filtered_rows = col('관광지 분류') == '인문관광지'

# # '무료'에 해당하는 키워드
# free_keywords = ['입 장 료:없음', '입 장 료:무료', '시설이용료:없음', '시설이용료:무료', '입장료 무료', '입장료 없음']

# # '유료'에 해당하는 키워드
# paid_keywords = [r'\d+천원', r'\d+원']

# # '유/무료 여부' 컬럼 설정
# df_fin = df_fin.withColumn('유/무료 여부',
#                           when(filtered_rows & col('상세정보').isNotNull() & col('상세정보').rlike('|'.join(free_keywords)), '무료')
#                           .when(filtered_rows & col('상세정보').isNotNull() & col('상세정보').rlike('|'.join(paid_keywords)), '유료')
#                           .otherwise('알 수 없음'))

In [16]:
# ### 3. 관광지분류 == '문화시설', '축제공연행사'

# # '문화시설' 및 '축제공연행사'에 해당하는 행 필터링
# filtered_rows = (col('관광지 분류').isin('문화시설', '축제공연행사'))

# # '무료'에 해당하는 키워드
# free_keywords = ['무료', '이용요금 없음']

# # '유료'에 해당하는 키워드
# paid_keywords = [r'\d+천원', r'\d+원', '유료']

# # '유/무료 여부' 컬럼 설정
# df_fin = df_fin.withColumn('유/무료 여부', 
#                           when(filtered_rows & col('이용요금').rlike('|'.join(free_keywords)), '무료')
#                           .when(filtered_rows & col('이용요금').rlike('|'.join(paid_keywords)), '유료')
#                           .otherwise('알 수 없음'))

### 📍 유모차 대여 여부 ['유모차 대여 여부'] : 없음/불가/가능

In [13]:
# 원본 데이터 값 확인
df_fin.select('유모차 대여 여부').distinct().show()

+----------------+
|유모차 대여 여부|
+----------------+
|            null|
|            없음|
|            불가|
|            가능|
+----------------+



In [14]:
# Transform
df_fin_replaced = df_fin.withColumn("유모차 대여 여부", 
                                    when((df_fin["유모차 대여 여부"].isNull()) | 
                                         (df_fin["유모차 대여 여부"] == "없음"), "불가")
                                    .otherwise(df_fin["유모차 대여 여부"]))

In [15]:
# 확인

## 불가, 가능으로 구성되어졌는지
df_fin_replaced.select('유모차 대여 여부').distinct().show()

## 갯수 맞게 들어갔는지 
### origin ver
tmp = df_fin.toPandas()
tmp['유모차 대여 여부'].value_counts(dropna=False)
### replaced ver
fin = df_fin_replaced.toPandas()
fin['유모차 대여 여부'].value_counts(dropna=False)

+----------------+
|유모차 대여 여부|
+----------------+
|            불가|
|            가능|
+----------------+



없음     2792
NaN     539
불가      248
가능       67
Name: 유모차 대여 여부, dtype: int64

불가    3579
가능      67
Name: 유모차 대여 여부, dtype: int64

### 📍 애완동물 동반 가능 여부 ['애완동물 동반 가능 여부'] : 없음/불가/가능

In [16]:
# 원본 데이터 값 확인
df_fin_replaced.select('애완동물 동반 가능 여부').distinct().show()

+-----------------------+
|애완동물 동반 가능 여부|
+-----------------------+
|                   null|
|                   없음|
|                   불가|
|                   가능|
+-----------------------+



In [17]:
# Transform
df_replaced = df_fin_replaced.withColumn("애완동물 동반 가능 여부", 
                                    when((df_fin["애완동물 동반 가능 여부"].isNull()) | 
                                         (df_fin["애완동물 동반 가능 여부"] == "없음"), "불가")
                                    .otherwise(df_fin["애완동물 동반 가능 여부"]))

In [18]:
# 확인

## 불가, 가능으로 구성되어졌는지
df_replaced.select('애완동물 동반 가능 여부').distinct().show()

## 갯수 맞게 들어갔는지 
### origin ver
tmp = df_fin.toPandas()
tmp['애완동물 동반 가능 여부'].value_counts(dropna=False)
### replaced ver
fin = df_replaced.toPandas()
fin['애완동물 동반 가능 여부'].value_counts(dropna=False)

+-----------------------+
|애완동물 동반 가능 여부|
+-----------------------+
|                   불가|
|                   가능|
+-----------------------+



불가     1416
없음     1133
가능      650
NaN     447
Name: 애완동물 동반 가능 여부, dtype: int64

불가    2996
가능     650
Name: 애완동물 동반 가능 여부, dtype: int64

## **4️⃣ 관람/체험 연령 관련 컬럼 전처리**

(총 데이터 갯수 3,648)
- [축제공연행사] 관람가능연령 : 7개 데이터
- [레포츠]
    - 체험가능연령 : 767 데이터, 그 중 70%가 *예약 관련 정보*로 들어가 있음
    - 주차시설 : 63 데이터, *연령 관련 정보가 이 컬럼으로 잘못 들어가 있음*
- [인문관광지] 체험가능연령 : 177개 데이터

➡️ '관람가능연령', '체험가능연령' 컬럼 제거 <br/>
➡️ [레포츠]에 해당되는 '주차시설'에 데이터 Null 처리

In [19]:
# "관람가능연령", "체험가능연령" 컬럼 drop
df_dropped = df_replaced.drop("관람가능연령", "체험가능연령")

In [20]:
# "관광지 분류"가 '레포츠' 인 데이터 중 "주차시설"인 컬럼 Null 값으로 바꾸기 
result_df = df_dropped.withColumn(
    "주차시설",
    when(col("관광지 분류") == "레포츠", lit(None)).otherwise(col("주차시설"))
)

In [21]:
# 확인

## Schema 확인 ("주차시설" 유지, "관람가능연령"&"체험가능연령" 삭제)
result_df.printSchema()

root
 |-- 개요: string (nullable = true)
 |-- 개장기간: string (nullable = true)
 |-- 공연시간: string (nullable = true)
 |-- 관람소요시간: string (nullable = true)
 |-- 명칭: string (nullable = true)
 |-- 분류: string (nullable = true)
 |-- 상세정보: string (nullable = true)
 |-- 쉬는날: string (nullable = true)
 |-- 애완동물 동반 가능 여부: string (nullable = true)
 |-- 우편번호: string (nullable = true)
 |-- 유모차 대여 여부: string (nullable = true)
 |-- 이용시간: string (nullable = true)
 |-- 이용시기: string (nullable = true)
 |-- 이용요금: string (nullable = true)
 |-- 입장료: string (nullable = true)
 |-- 전화번호: string (nullable = true)
 |-- 주소: string (nullable = true)
 |-- 주차시설: string (nullable = true)
 |-- 주차요금: string (nullable = true)
 |-- 지역: string (nullable = true)
 |-- 체험안내: string (nullable = true)
 |-- 행사시작일: long (nullable = true)
 |-- 행사종료일: long (nullable = true)
 |-- 관광지 분류: string (nullable = true)



In [22]:
## 갯수 맞게 들어갔는지 
### origin ver
origin = df_replaced.toPandas()
tmp = origin[origin['관광지 분류'] == '레포츠']
print('origin 레포츠 주차시설 데이터 갯수: ', tmp['주차시설'].count())

### replaced ver
result = result_df.toPandas()
tmp2 = result[result['관광지 분류'] == '레포츠']
print('replaced 레포츠 주차시설 데이터 갯수: ', tmp2['주차시설'].count())
print('전처리 한 주차시설 데이터 확인: ')
result['주차시설'].value_counts(dropna=False)

origin 레포츠 주차시설 데이터 갯수:  63
replaced 레포츠 주차시설 데이터 갯수:  0
전처리 한 주차시설 데이터 확인: 


NaN                                                      1615
있음                                                        378
가능                                                        227
가능<br>요금(무료)                                              165
주차가능                                                      154
                                                         ... 
있음(3개소, 661대 주차 가능)                                         1
있음 (모음동 소형 31대 / 나눔동 소형 4대 / 배움동 소형 6대 / 부설주차장 / 13대)       1
있음(대소형 약1,500대)                                             1
있음(총 170면)                                                  1
가능(몽골문화촌 주차장)<br>요금(무료)                                     1
Name: 주차시설, Length: 699, dtype: int64

## **5️⃣ 명칭 컬럼 Null 데이터 제거**

In [23]:
result_df.printSchema()

root
 |-- 개요: string (nullable = true)
 |-- 개장기간: string (nullable = true)
 |-- 공연시간: string (nullable = true)
 |-- 관람소요시간: string (nullable = true)
 |-- 명칭: string (nullable = true)
 |-- 분류: string (nullable = true)
 |-- 상세정보: string (nullable = true)
 |-- 쉬는날: string (nullable = true)
 |-- 애완동물 동반 가능 여부: string (nullable = true)
 |-- 우편번호: string (nullable = true)
 |-- 유모차 대여 여부: string (nullable = true)
 |-- 이용시간: string (nullable = true)
 |-- 이용시기: string (nullable = true)
 |-- 이용요금: string (nullable = true)
 |-- 입장료: string (nullable = true)
 |-- 전화번호: string (nullable = true)
 |-- 주소: string (nullable = true)
 |-- 주차시설: string (nullable = true)
 |-- 주차요금: string (nullable = true)
 |-- 지역: string (nullable = true)
 |-- 체험안내: string (nullable = true)
 |-- 행사시작일: long (nullable = true)
 |-- 행사종료일: long (nullable = true)
 |-- 관광지 분류: string (nullable = true)



In [24]:
result_dropped = result_df.dropna(subset=["명칭"])

In [25]:
tmpp = result_dropped.toPandas()
tmpp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3644 entries, 0 to 3643
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   개요             3641 non-null   object 
 1   개장기간           583 non-null    object 
 2   공연시간           33 non-null     object 
 3   관람소요시간         245 non-null    object 
 4   명칭             3644 non-null   object 
 5   분류             3644 non-null   object 
 6   상세정보           3162 non-null   object 
 7   쉬는날            2392 non-null   object 
 8   애완동물 동반 가능 여부  3644 non-null   object 
 9   우편번호           3532 non-null   object 
 10  유모차 대여 여부      3644 non-null   object 
 11  이용시간           2790 non-null   object 
 12  이용시기           12 non-null     object 
 13  이용요금           680 non-null    object 
 14  입장료            24 non-null     object 
 15  전화번호           107 non-null    object 
 16  주소             3642 non-null   object 
 17  주차시설           2031 non-null   object 
 18  주차요금    

## **6️⃣ 불필요한 데이터 제거**
- 아래 데이터 모두 제거
    - '명칭'&'주소' 기준으로 중복으로 들어간 데이터
    - 이미지 파일이 없는 데이터
    - 특수기호가 포함되어 이미지 모델링에 적용되지 않는 데이터

In [26]:
# 중복된 데이터 찾기
## '명칭'이 같은 관광지가 있을 수 있어, '주소' 컬럼과 같이 groupBy하여 찾기 
duplicate_rows = result_dropped.groupBy('명칭', '주소').count().where("count > 1")
duplicate_rows.show()

+------------------+--------------------------------+-----+
|              명칭|                            주소|count|
+------------------+--------------------------------+-----+
|안산세계문화체험관|     경기도 안산시 단원구 원곡동|    3|
|          수연목서|         경기도 여주시 주어로 58|    2|
|      중남미문화원|경기도 고양시 덕양구 대양로28...|    3|
+------------------+--------------------------------+-----+



In [27]:
# 중복된 데이터 제거
unique_df = result_dropped.dropDuplicates(['명칭', '주소'])

In [28]:
# 확인 
## 중복된 데이터 잘 제거되었는지
duplicate_rows = unique_df.groupBy('명칭', '주소').count().where("count > 1")
duplicate_rows.show()

+----+----+-----+
|명칭|주소|count|
+----+----+-----+
+----+----+-----+



In [29]:
# 이미지 파일이 없는 데이터 제거 (1)

remove_list = ['SPA 1899 동인비', '가막들공원', '가평사계절썰매장', '간데메공원', '강남 마이스 관광특구', '강화나들 8코스', '건지공원',
 '고천체육공원', '곤충도시', '공릉호', '글로우사파리', '금빛공원', '금천 체육공원', '금천 폭포공원', '긴고랑 공원',
 '낙산공원', '남동공단근린공원', '내손체육공원', '노들나루공원', '노작공원', '늘솔길공원', '다락원체육공원', '달맞이광장',
 '달맞이봉공원', '당남지구공원', '더 스파 앳 파라다이스', '동막골유원지', '동인천역북광장', '마리앤쥬', '마이랜드',
 '마천공원', '만남의광장', '만해기념관', '묵정어린이공원', '미사한강공원4호', '반포한강공원 달빛광장', '방배배수지체육공원',
 '방화근린공원', '부곡체육공원', '부천역마루광장', '비스타밸리', '산악자전거 자운리코스', '삼목선착장', '새아침공원',
 '서수원체육공원', '서울색공원', '서울어린이대공원', '서울함공원', '선두바다낚시터', '설화수 플래그십 스토어', '성사얼음마루',
 '성촌공원', '세빛섬', '솔찬공원', '송우관광농원 페리아도 워터파크', '송파구자원순환공원', '스트레칭조이', '스파어바인',
 '신사공원', '씨사이드파크하늘구름광장', '씨사이드파크해수족욕장', '아르티스 테마파크', '아이스하우스 아이스링크', '아쿠아필드 고양',
 '양주 나리공원', '여의도한강공원 멀티프라자', '여주강변유원지', '여주도자세상', '역곡공원', '역삼개나리공원', '영흥면에너지파크',
 '오이도 선사유적공원', '올리바인 스파', '올림픽공원피크닉장', '용마공원놀이동산(용마랜드)', '용산공원 (옛 미군기지)', '용산역광장',
 '용인 곤충테마파크', '우장산공원', '원마운트 워터파크', '원주 산악자전거파크', '월드컵공원', '월문온천 휴양지', '유엔군 초전기념관',
 '윤동주시인의언덕', '율암온천숯가마 테마파크', '율현공원', '은가어린이공원', '은곡마을공원', '응봉체육공원', '인더쥬동물원',
 '인천항 국제여객터미널', '인천항 크루즈터미널', '일원장미공원', '잠실유수지공원', '장안근린공원', '주암체육소공원', '주인근린공원',
 '청계광장', '청라 스파렉스', '코엑스동측광장', '테르메덴', '파라다이스 씨티 씨메르(인천)', '파라다이스시티', 
 '파라다이스시티 원더박스', '파주가야랜드', '펄벅기념관', '평화광장', '평화의공원', '평화잔디광장', '플레이 아쿠아리움 부천',
 '하남아이스링크', '하늘체육공원', '하니랜드', '한국민속촌 눈썰매장', '한방스파 여용국', '한솔공원', '한수공원', '한양공원',
 '해수워터피아', '헬로애니멀', '현대원서공원', '홍익문화공원', '화랑대 철도공원', '화성식염온천', '황금내근린공원', 
 '서울국제가구 및 인테리어산업대전 : 소펀&라이프쇼', '연평도(대/소 연평도)', '이포보오토캠핑장_양촌지구', '이포보웰빙캠핑장_담낭지구',
 '모나캠핑파크/독박골대청마루', '한국국제가구 및 인테리어산업대전 : 코펀', '<청와대 국민과 함께> 영상전시']

df_filtered = unique_df.filter(~col('명칭').isin(remove_list))

In [30]:
# 확인 
print('이미지 파일이 없는 데이터 갯수: ', len(remove_list))

이미지 파일이 없는 데이터 갯수:  133


In [31]:
# 이미지 파일이 없는 데이터 제거 (2)
df_tmp = df_filtered.filter(~((col('명칭') == '마루공원') & (col('주소') == '서울특별시 강남구 개포로109길 74(개포동)')))
df_filtered_fin = df_tmp.filter(~((col('명칭') == '중앙공원') & (col('주소') == '경기도 군포시 광정로 96(산본동)')))

In [32]:
# 확인 
## 이미지 데이터 잘 제거되었는지
df_filtered_fin.select('명칭', '주소').where(col('명칭')=='마루공원').show() # 성남인 데이터는 살아 있어야 함 
df_filtered_fin.select('명칭', '주소').where(col('명칭')=='중앙공원').show() # 인천인 데이터는 살아 있어야 함 

+--------+--------------------------------+
|    명칭|                            주소|
+--------+--------------------------------+
|마루공원|경기도 성남시 분당구 수내로 2...|
+--------+--------------------------------+

+--------+-------------------------------+
|    명칭|                           주소|
+--------+-------------------------------+
|중앙공원|인천광역시 남동구 예술로 125...|
+--------+-------------------------------+



In [33]:
# 최종확인
## 총 데이터 갯수 : 3644(기존 데이터 갯수) - 5 (중복된 데이터 갯수) - 135(이미지 없는 데이터 갯수) = 3504개가 나와야 함
tmp = df_filtered_fin.toPandas()
tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3504 entries, 0 to 3503
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   개요             3501 non-null   object 
 1   개장기간           577 non-null    object 
 2   공연시간           30 non-null     object 
 3   관람소요시간         240 non-null    object 
 4   명칭             3504 non-null   object 
 5   분류             3504 non-null   object 
 6   상세정보           3038 non-null   object 
 7   쉬는날            2304 non-null   object 
 8   애완동물 동반 가능 여부  3504 non-null   object 
 9   우편번호           3400 non-null   object 
 10  유모차 대여 여부      3504 non-null   object 
 11  이용시간           2678 non-null   object 
 12  이용시기           11 non-null     object 
 13  이용요금           670 non-null    object 
 14  입장료            22 non-null     object 
 15  전화번호           104 non-null    object 
 16  주소             3502 non-null   object 
 17  주차시설           1933 non-null   object 
 18  주차요금    

## **7️⃣ 인덱스 생성**

In [ ]:
# df_with_index = df_filtered_fin.toPandas()

In [ ]:
# df_with_index.reset_index(inplace=True)
# df_with_index.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [ ]:
# df_with_index['index'] = df_with_index['index']+1

In [ ]:
# df_with_index.head(2)
# df_with_index.info()

In [38]:
######## 맞춰야 할 index ########
# check_index   : pd
# df_idx   : spark

In [34]:
# df_idx = spark.read.json('/raw_data/travel_data_idx.json', encoding='utf-8')

In [35]:
# check_index = df_idx.toPandas()

In [43]:
# 확인
# check_index[check_index['index']==3504]

,Unnamed: 0,index,개요,개장기간,공연시간,관광지 분류,관람소요시간,명칭,분류,상세정보,...,이용요금,입장료,전화번호,주소,주차시설,주차요금,지역,체험안내,행사시작일,행사종료일
3503,3504,3504,경기도에서 정식 온천으로 허가받은 자연 용출수 온천을 이용할 수 있다. 이곳에서 사...,None,None,인문관광지,None,힐사이드온천텔,휴양관광지,"입 장 료:- 대실 25,000원~40,000원<br>\n- 숙박 40,000원~1...",...,None,None,None,경기도 화성시 팔탄면 온천로 428-12,주차가능,None,경기도,None,NaN,NaN


In [45]:
# check_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3504 entries, 0 to 3503
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     3504 non-null   int64  
 1   index          3504 non-null   int64  
 2   개요             3501 non-null   object 
 3   개장기간           577 non-null    object 
 4   공연시간           30 non-null     object 
 5   관광지 분류         3504 non-null   object 
 6   관람소요시간         240 non-null    object 
 7   명칭             3504 non-null   object 
 8   분류             3504 non-null   object 
 9   상세정보           3038 non-null   object 
 10  쉬는날            2304 non-null   object 
 11  애완동물 동반 가능 여부  3504 non-null   object 
 12  우편번호           3400 non-null   object 
 13  유모차 대여 여부      3504 non-null   object 
 14  이용시간           2678 non-null   object 
 15  이용시기           11 non-null     object 
 16  이용요금           670 non-null    object 
 17  입장료            22 non-null     object 
 18  전화번호    

In [46]:
# check_index.drop(columns=['Unnamed: 0'], inplace=True)

In [47]:
# check_index.head()

,index,개요,개장기간,공연시간,관광지 분류,관람소요시간,명칭,분류,상세정보,쉬는날,...,이용요금,입장료,전화번호,주소,주차시설,주차요금,지역,체험안내,행사시작일,행사종료일
0,1,"교보문고 광화문점은 국민교육 진흥의 실천적 구현, 독서인구 저변 확대를 통한 국민정...",None,None,문화시설,None,(주)교보문고,문화시설,None,설/추석 당일,...,무료,None,None,서울특별시 종로구 종로 1(종로1가),주차 가능,유료 (회원여부/구매금액에 따라 무료주차헤택 적용),서울,None,NaN,NaN
1,2,※ 민통초소에 신분증 제시하여 출입 가능 (25인 이상 출입시 일주일 전 신청서 제...,None,None,인문관광지,None,1.21 무장공비 침투로,역사관광지,화장실:있음\n,매주 화요일,...,None,None,None,경기도 연천군 장남면 반정리 294,None,None,경기도,None,NaN,NaN
2,3,코리아그랜드세일은 관광과 한류가 융복합된 외국인 대상의 한국 대표 쇼핑문화관광축제로...,None,None,축제공연행사,None,2024 코리아그랜드세일,공연행사,행사소개:코리아그랜드세일은 관광과 한류가 융복합된 외국인 대상의 한국 대표 쇼핑문화...,None,...,무료,None,070-7787-4242,서울특별시 종로구 인사동5길 29 태화빌딩802호,None,None,서울,None,20240111.0,20240229.0
3,4,"경기도 화성시 서신면에 위치해 있는 24시 마린 낚시는 주변에 전곡항, 탄도항, 제...",None,None,레포츠,None,24시 마린낚시(전곡항 24 마린낚시),레포츠,"포인트안내:\n이용요금:선박별 상이 홈페이지 참조\n보유장비: 22인승, 21인승,...",None,...,None,None,None,경기도 화성시 서신면 전곡항로 317,None,None,경기도,None,NaN,NaN
4,5,그림을 열망하던 청년은 첫 월급 3000원을 쪼개 그림 한 점을 구입했다. 당시 가...,None,None,문화시설,None,93뮤지엄,문화시설,None,"매주 월요일(단,월요일이 공휴일인 경우 정상 개관,해당주 화요일 휴관)",...,기간에따라 변동,None,None,경기도 파주시 탄현면 헤이리마을길 59-58헤이리예술마을 G-31,주차가능,None,경기도,None,NaN,NaN


In [48]:
# 다시 spark df 로 

# # 'Attribute Error: 'iteritems'가 없다고 나올때
# import pandas as pd
# pd.DataFrame.iteritems = pd.DataFrame.items

spark_df = spark.createDataFrame(check_index)

In [49]:
type(spark_df)

pyspark.sql.dataframe.DataFrame

## **파일 저장**

In [47]:
# csv, xlsx 파일로 저장
# df_with_index.to_csv('./travel_data_idx.csv')
# df_with_index.to_excel('./travel_data_idx.xlsx')

In [50]:
# hdfs에 저장
spark_df.coalesce(1).write.format('json').save('/dw_data/travel_data_preprocessed.json')